In [ ]:
# =======================================================
# EXPORT4PHOBS — Adria Ankaran PHOBS BAR Export Tool
# Ready for Voila deployment
# =======================================================

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output, IFrame
from datetime import datetime
import base64
import urllib.parse
import time


# =======================================================
# 3️⃣ Header
# =======================================================
header_html = """
<div style="display:flex; align-items:center; justify-content:space-between; margin-bottom:10px;">
    <div>
        <h2 style="margin:0; color:#004080;"><b>EXPORT</b>4PHOBS</h2>
        <p style="font-size:14px; color:gray; margin-top:4px;">Adria Ankaran PHOBS Export Dashboard</p>
    </div>
    <img src="https://www.adria-ankaran.si/app/uploads/2025/10/logo-Adria.jpg" width="150">
</div>
<hr>
"""
display(HTML(header_html))

# =======================================================
# 1️⃣ Load Custom CSS (Branding)
# =======================================================
display(HTML('<link rel="stylesheet" type="text/css" href="custom.css">'))

# =======================================================
# 2️⃣ Display Google Sheet IFrame
# =======================================================
display(IFrame(
    src='https://docs.google.com/spreadsheets/d/15HJ7wxyUmo-gcl5_y1M9gl4Ti-JSsYEJZCjoI76s-Xk/edit?gid=1385640257#gid=1385640257',
    width=1200,
    height=600
))

# =======================================================
# 4️⃣ Reload Button + Timestamp
# =======================================================
status_out = widgets.Output()
reload_btn = widgets.Button(
    description="🔄 Reload Hotels",
    button_style='warning',
    layout=widgets.Layout(width='200px')
)
status_box = widgets.HBox([reload_btn, status_out])
display(status_box)

def update_status():
    with status_out:
        clear_output()
        now = datetime.now().strftime("%H:%M:%S")
        display(HTML(f"<b>Last refreshed:</b> {now}"))

# =======================================================
# 5️⃣ Prepare PHOBS CSV Function
# =======================================================
def prepare_phobs_csv(df, hotel_id, los_code):
    df = df.copy()
    if 'BAR' not in df.columns:
        df['BAR'] = 120
    df['BAR'] = df['BAR'].apply(lambda x: f"BAR{x}")
    df['Hotel_ID'] = hotel_id
    df['nicla'] = 0
    df['Yield'] = f"YIELD{los_code}"
    if 'Datum' not in df.columns:
        df['Datum'] = pd.Timestamp.today().strftime('%Y-%m-%d')
    df = df[['Hotel_ID', 'Datum', 'nicla', 'BAR', 'Yield']]
    return df

# =======================================================
# 6️⃣ Create Download Button Function
# =======================================================
download_out = widgets.Output()

def create_download_button(df, filename, label):
    btn = widgets.Button(
        description=f"📥 {label}",
        button_style='info',
        layout=widgets.Layout(width='260px', margin='4px')
    )
    def on_click(b):
        csv_bytes = df.to_csv(index=False, header=False).encode()
        b64 = base64.b64encode(csv_bytes).decode()
        href = f"data:text/csv;base64,{b64}"
        html = f"""
        <div style='margin-top:10px;'>
            <a download="{filename}.csv" href="{href}" target="_blank"
               style='text-decoration:none; color:white; background-color:#0078D7; padding:6px 12px; border-radius:4px;'>
               📄 Download {filename}.csv
            </a>
        </div>
        """
        with download_out:
            clear_output()
            display(HTML(html))
    btn.on_click(on_click)
    return btn

# =======================================================
# 7️⃣ Progress Bar & Hotel Loading
# =======================================================
progress = widgets.IntProgress(
    value=0, min=0, max=100, step=1,
    bar_style='', orientation='horizontal',
    layout=widgets.Layout(width='60%')
)
progress_label = widgets.Label("")
progress_box = widgets.HBox([progress, progress_label])
display(progress_box)

# =======================================================
# 8️⃣ Load Hotels and Generate Buttons
# =======================================================
gsheet_id = "15HJ7wxyUmo-gcl5_y1M9gl4Ti-JSsYEJZCjoI76s-Xk"
master_sheet_name = "PHOBS"

content_out = widgets.Output()
display(content_out)

def load_hotels():
    update_status()
    with content_out:
        clear_output()
        display(HTML("<p style='color:gray;'>⏳ Loading hotel data from Google Sheets...</p>"))
        try:
            master_url = f"https://docs.google.com/spreadsheets/d/{gsheet_id}/gviz/tq?tqx=out:csv&sheet={master_sheet_name}"
            master_df = pd.read_csv(master_url)
            hotels = [(row['Hotel_Name'], row['Hotel_ID'], row['YIELD_Code'])
                      for _, row in master_df.iterrows()]

            total_hotels = len(hotels)
            progress.max = total_hotels
            progress.value = 0
            progress.bar_style = ''
            progress_label.value = f"Loading 0/{total_hotels} hotels..."

            buttons = []

            for idx, (hotel_name, hotel_id, los_code) in enumerate(hotels, start=1):
                try:
                    sheet_name = urllib.parse.quote(hotel_name)
                    sheet_url = f"https://docs.google.com/spreadsheets/d/{gsheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
                    df = pd.read_csv(sheet_url)
                    df = prepare_phobs_csv(df, hotel_id, los_code)
                    button = create_download_button(df, f"{hotel_name}-PHOBS", hotel_name.replace("_", " "))
                    buttons.append(button)
                except Exception as e:
                    display(HTML(f"<div style='color:red;'>⚠️ <b>{hotel_name}</b> failed to load: {e}</div>"))

                progress.value = idx
                progress_label.value = f"Loaded {idx}/{total_hotels} hotels"
                time.sleep(0.1)

            progress.bar_style = 'success'
            clear_output(wait=True)
            display(HTML("<h3 style='color:#004080;'>✅ PHOBS BAR Export Generator</h3>"))

            if buttons:
                rows = [widgets.HBox(buttons[i:i+3]) for i in range(0, len(buttons), 3)]
                vbox = widgets.VBox(rows)
                display(vbox, download_out)
            else:
                display(HTML("<b style='color:red;'>No hotel data found.</b>"))

        except Exception as e:
            clear_output()
            display(HTML(f"<b style='color:red;'>Error loading master sheet:</b> {e}"))

# Bind reload button
reload_btn.on_click(lambda b: load_hotels())

# Initial load
load_hotels()
